<a href="https://colab.research.google.com/github/katduecker/dnn_osci/blob/main/Thesis_ch3_one_layer_temporal_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/katduecker/dnn_osci
%cd dnn_osci
%cd aet_pytorch/

Cloning into 'dnn_osci'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 454 (delta 18), reused 6 (delta 2), pack-reused 413
Receiving objects: 100% (454/454), 36.08 MiB | 21.74 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/dnn_osci/aet_pytorch/dnn_osci
/content/dnn_osci/aet_pytorch/dnn_osci/aet_pytorch


In [21]:
import torch
import matplotlib.pyplot as plt
from torch import nn
import numpy as np

import aet_dyn
import aet_net
# get CUDA index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
def euler_dyn(model,input_,params_,t_,alpha_params,DEVICE,inp_on):

    # inputs:
        # input_: input image
        # params_: hyperparameters
        # t_: time vector for integration
        # alpha_params: alpha frequency & amplitude

    # discretization & dynamics parameters
    tau_h,tau_R,R,T,h_start,R_start = params_


    # alpha frequency & amplitude
    _af,_aa = alpha_params

    with torch.no_grad():

        # initialize empty matrices
        dt = np.diff(t_)[0]
        dhdt = (torch.ones((model.dims[1],len(t_)+1))*h_start).to(DEVICE)
        dRdt = (torch.ones((model.dims[1],len(t_)+1))*R_start).to(DEVICE)
        dOdt = (torch.zeros((model.dims[2],len(t_)+1))).to(DEVICE)
        dZdt = (torch.zeros((model.dims[1],len(t_)+1))).to(DEVICE)

        # alpha inhibition
        alpha_inh = _aa*np.sin(2*np.pi*_af*t_)+_aa

        # preactivation (dot product of input and first weight matrix)
        Z,_,_ = model.forw_conv(input_)

        # create boxcar function to try different input onsets
        boxcar = np.zeros_like(t_)
        boxcar[inp_on:] = 1

        # adjust initial adaptation term (threshold)
        dRdt *= torch.max(Z)

        # scale for adaptation
        #r_scale = R*torch.max(Z).detach()


        for _it,t in enumerate(t_):

            # dynamic input: multiply input with boxcar
            Z,_,_ = model.forw_conv(input_*boxcar[_it])

            # pre-activation
            dZdt[:,_it+1] = (Z + dhdt[:,_it] - dRdt[:,_it] - alpha_inh[_it])/T

            # dynamics hidden layer
            dhdt[:,_it+1] = dhdt[:,_it] + dt/tau_h * (-dhdt[:,_it] + model.acti1(dZdt[:,_it+1],model.sig_param))

            # adaptation term
            dRdt[:,_it+1] = dRdt[:,_it] + dt/tau_R * (-dRdt[:,_it] + R*np.mean(dZdt[:,_it+1])*dhdt[:,_it+1])

            # output layer
            dOdt[:,_it+1] = model.actiout(model.fc1(dhdt[:,_it+1]))

    return dZdt, dhdt, dRdt, dOdt


In [23]:
nn_dim_ = [28,68,3]   # [quadrant size, number of hidden nodes, number of output nodes]
eta_ = 0.1           # learning rate
mini_sz_ = 1          # mini batch size (1 = use SGD)
num_epo_ = 80

beta_ = 0
p_ = 0
kl_reg_ = [beta_,p_]#[0,0.001] # sparsity constraint parameters (not used for manual model)
sig_param = [2, -2.5] # sigmoid slope and shift in x direction

# loss function & final layer activation (for binary crossentropy use sigmoid)
lossfun = [nn.MSELoss(), nn.Softmax(dim=-1)]

params = nn_dim_,eta_,mini_sz_,num_epo_,kl_reg_,sig_param

# initialize model and weights
model = aet_net.net(params,lossfun)
model = aet_net.init_params(model,weight_init='uni')
optimizer = torch.optim.SGD(model.parameters(),lr=eta_)

model.to(DEVICE)
loss_hist = model.train(optimizer,noise=False,print_loss=False)


plt.rcParams["figure.figsize"] = (10,5)
plt.plot(np.arange(model.num_ep),loss_hist.cpu().detach().numpy())
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AttributeError: ignored

In [ ]:
if model.reg[0]:
  print('h')